In [1]:
import os
os.chdir(os.path.join(os.getcwd(), '..'))
os.getcwd()

'/home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA'

In [2]:
from src.utils import get_dataset
from src.utils import select_data

import scipy.io as sio
from scipy.signal import resample
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

from keras.utils import to_categorical
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Bidirectional
from keras.layers import Dropout
from keras.layers import Conv1D
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint

/home/renjie/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
DATA_PATH = os.path.join(os.getcwd(), 'data')
os.listdir(DATA_PATH)

['Depth', 'Inertial', 'Skeleton', 'RGB']

In [4]:
depth_path, inertial_path, skeleton_path,rgb_path = get_dataset(DATA_PATH)

In [5]:
activities = list(range(1,28))

In [6]:
def select_subject(d_path, subject):
    select_statement = '_s{}_'.format(subject)
    subjects = []
    for i in d_path:
        if select_statement in i:
            subjects.append(i)
    return subjects

In [7]:
def get_action_number(single_path):
    return int(single_path.split('/')[-1].split('_')[0][1:])
def get_subject_number(single_path):
    return int(single_path.split('/')[-1].split('_')[1][1:])
def get_trial_number(single_path):
    return int(single_path.split('/')[-1].split('_')[2][1:])


In [8]:
def pad_len_inertial(inertial_d):
    inertial_d = np.swapaxes(inertial_d, 0,1)
    inertial_d = sequence.pad_sequences(inertial_d, maxlen=326)
    inertial_d = np.swapaxes(inertial_d, 0,1)
    return inertial_d

In [9]:
# train on subject 1,3,5,7
# test on subject 2,4,6,8
X_train = []
Y_train = []
X_test = []
Y_test = []

resample_len = 180

for path in inertial_path:
    if get_subject_number(path) in [1,3,5,7]:
        X_train.append(path)
        Y_train.append(get_action_number(path))
    else:
        X_test.append(path)
        Y_test.append(get_action_number(path))

# X_train = [pad_len_inertial(sio.loadmat(x)['d_iner']) for x in X_train]
# X_test = [pad_len_inertial(sio.loadmat(x)['d_iner']) for x in X_test]

X_train = [resample(sio.loadmat(x)['d_iner'], resample_len) for x in X_train]
X_test = [resample(sio.loadmat(x)['d_iner'], resample_len) for x in X_test]


X_train = np.array(X_train)
X_test = np.array(X_test)

Y_train = to_categorical(np.array(Y_train) - 1)
Y_test = to_categorical(np.array(Y_test) - 1)

X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((431, 180, 6), (431, 27), (430, 180, 6), (430, 27))

# Model

In [10]:
import keras.backend as K


def f1(y_true, y_pred):
    '''
    Calculates the F1 by using keras.backend
    '''
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))

    def recall(y_true, y_pred):
        """Recall metric.
        Only computes a batch-wise average of recall.
        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.
        Only computes a batch-wise average of precision.
        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
#     print("precision: ", precision)
#     print("recall: ", recall)
    return 2 * ((precision * recall) / (precision + recall))

In [ ]:
def simple_LSTM():
    np.random.seed(99)
    model = Sequential(name = 'simple_LSTM')
    model.add(LSTM(512, input_shape=(None, 6), recurrent_dropout=0.5))
    model.add(Dense(len(activities), activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-3), metrics=['accuracy', 'mse',f1])
    print(model.summary())
    return model

In [ ]:
def bidirectional_LSTM():
    np.random.seed(7)
    model = Sequential(name = 'bidirectional_LSTM')
    model.add(Bidirectional(LSTM(50), input_shape=(None, 6), recurrent_dropout=0.5))
    model.add(Dense(len(activities), activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-3), metrics=['accuracy', 'mse',f1])
    print(model.summary())
    return model

In [16]:
def conv_LSTM():
    # Create the model
    np.random.seed(7)
    optimizer = Adam(lr=1e-4, decay=1e-6, clipnorm=0.6)
    model = Sequential(name = 'conv_LSTM')
    model.add(Conv1D(128,
                     4,
                     padding='valid',
                     activation='relu',
                     strides=1,
                    input_shape=(180, 6)))
    model.add(Conv1D(64,
                     4,
                     padding='valid',
                     activation='relu',
                     strides=1))
    #model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(64,
                     4,
                     padding='valid',
                     activation='relu',
                     strides=1))
    model.add(Conv1D(64,
                     4,
                     padding='valid',
                     activation='relu',
                     strides=1))
    model.add(LSTM(256, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
    model.add(LSTM(256, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
    model.add(Flatten())
    #model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(len(activities), activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    print(model.summary())
    return model

In [17]:
# model = simple_LSTM()
# model = bidirectional_LSTM()
model = conv_LSTM()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_5 (Conv1D)            (None, 177, 128)          3200      
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 174, 64)           32832     
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 171, 64)           16448     
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 168, 64)           16448     
_________________________________________________________________
lstm_3 (LSTM)                (None, 168, 256)          328704    
_________________________________________________________________
lstm_4 (LSTM)                (None, 168, 256)          525312    
_________________________________________________________________
flatten_2 (Flatten)          (None, 43008)             0         
__________

In [18]:
LOG_DIR = os.path.join(os.getcwd(), 'logs')
tb = TensorBoard(LOG_DIR)

In [19]:
weights_dir = 'weights/' + model.name + \
            '-{epoch:02d}-{loss:.2f}.hdf5'
chkpt = ModelCheckpoint(filepath=weights_dir, monitor='loss', save_best_only=True, save_weights_only=True, mode='auto', period=1)

In [21]:
history = model.fit(X_train, Y_train, epochs=15, batch_size=4, validation_data = (X_test, Y_test), callbacks=[tb, chkpt])

Train on 431 samples, validate on 430 samples
Epoch 1/15
428/431 [============================>.] - ETA: 0s - loss: 0.0202 - acc: 0.9977

KeyboardInterrupt: 

# Evaluation

In [ ]:
train_acc = history.history['acc']
val_acc = history.history['val_acc']
plt.plot(train_acc, 'C0')
plt.plot(val_acc, 'C1')
plt.show()

In [ ]:
train_acc = history.history['loss']
val_acc = history.history['val_loss']
plt.plot(train_acc, 'C0')
plt.plot(val_acc, 'C1')
plt.show()

In [ ]:
model.evaluate(X_train, Y_train)

In [ ]:
model.evaluate(X_test, Y_test)

In [ ]:
pred_1 = np.argmax(model.predict(X_test), axis = -1)

In [ ]:
class_labels = ['swipe to the left',
               'swipe to the right',
               'wave',
               'front clap',
               'throw',
               'cross arms',
               'basketball shoot',
               'draw x',
               'draw circle (CW)',
               'draw circle (CCW)',
               'draw triangle',
               'bowling',
               'boxing',
               'baseball swing',
               'tennis swing',
               'arm curl',
               'tennis serve',
               'two hand push',
               'knock door',
               'catch',
               'pick and throw',
               'jogging',
               'walking',
               'sit to stand',
               'stand to sit',
               'forward lunge',
               'squat']

In [ ]:
confusion_1 = confusion_matrix(np.argmax(Y_test, axis = -1), pred_1)
NUM_LABELS = 27

f, axes = plt.subplots(1,1, figsize = (12,12))
axes.set_xlabel('Actual')
axes.set_ylabel('Predicted')
axes.grid(False)
axes.set_xticklabels(class_labels, rotation = 90)
axes.set_yticklabels(class_labels)
axes.set_yticks(list(range(27)))
axes.set_xticks(list(range(27)))
plt.imshow(confusion_1, cmap=plt.cm.Set2, interpolation='nearest')

for i, cas in enumerate(confusion_1):
    for j, count in enumerate(cas):
        if count > 0:
            xoff = .07 * len(str(count))
            plt.text(j-xoff, i+.2, int(count), fontsize=12, color='black')

# Seeing if there is a bug

In [ ]:
for i in range(len(inertial_path)):
    test = sio.loadmat(inertial_path[i])['d_iner']
    test = np.expand_dims(np.swapaxes(sequence.pad_sequences(np.swapaxes(test,1,0), maxlen= 326), 1, 0), axis =0)
    pred = model.predict(test)
    print('Predicted: {} Expected: [{}]'.format(np.argmax(pred, axis = -1) + 1, get_action_number(inertial_path[i])))

In [ ]:
num = 810

In [ ]:
test = sio.loadmat(inertial_path[num])['d_iner']

In [ ]:
test = np.expand_dims(np.swapaxes(sequence.pad_sequences(np.swapaxes(test,1,0), maxlen= 326), 1, 0), axis =0)

In [ ]:
pred = model.predict(test)

In [ ]:
np.argmax(pred, axis = -1), inertial_path[num].split('/')[-1]